# Global Structural Integrity Production (One-Shot)

This notebook generates a global structural integrity layer using the quality-weighted core area approach with a **single export task**.

**Methodology:**
- Binary habitat mask from HMI (threshold < 0.4 = natural/semi-natural)
- Erosion by edge depth to identify core habitat
- Weight core pixels by quality class (pristine=4, low-impact=3, moderate=2, semi-natural=1)
- Calculate weighted mean within neighborhood, normalize to 0-1

**Advantages of one-shot approach:**
- Single export task (vs ~260 tiled tasks)
- GEE handles internal tiling automatically
- Simpler monitoring and error handling
- Single output asset for easy loading

In [ ]:
import datetime

import ee

ee.Initialize()

from eii.compute.settings import SPATIAL_RESOLUTION
from eii.compute.structural import calculate_structural_integrity
from eii.utils.gee import create_assets_folder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Asset Root: projects/landler-open-data/assets/eii
Model Version: v1


## Configuration

In [ ]:
# Output asset path
OUTPUT_ASSET_PATH = (
    "projects/landler-open-data/assets/eii/products/v1/structural_integrity/core_area"
)

# Structural integrity parameters (from eii.compute.structural defaults)
EDGE_DEPTH_M = 300  # Edge effect penetration depth
NEIGHBORHOOD_M = 5000  # Landscape analysis radius
SCALE_M = SPATIAL_RESOLUTION  # Export resolution (300m)

# Global bounds (exclude polar regions)
GLOBAL_BOUNDS = ee.Geometry.Rectangle([-180, -60, 180, 80], proj="EPSG:4326", geodesic=False)

# Ensure parent folder exists
parent_folder = "/".join(OUTPUT_ASSET_PATH.split("/")[:-1])
create_assets_folder(parent_folder)

print(f"Output Asset: {OUTPUT_ASSET_PATH}")
print(f"Edge Depth: {EDGE_DEPTH_M}m")
print(f"Neighborhood Radius: {NEIGHBORHOOD_M}m")
print(f"Export Scale: {SCALE_M}m")

Created: projects/landler-open-data/assets/eii/products/v1/structural_integrity/core_area
Edge Depth: 300m
Neighborhood Radius: 5000m
Export Scale: 300m
Tile Size: 10.0°
Latitude Range: -60° to 80°
Output Asset Path: projects/landler-open-data/assets/eii/products/v1/structural_integrity/core_area
Grid Asset Path: projects/landler-open-data/assets/eii/intermediate/functional/grid


## Step 1: Build Global Structural Integrity

Using `calculate_structural_integrity(aoi=None)` returns global unclipped image.

In [ ]:
# Build global structural integrity (aoi=None returns unclipped global image)
structural_integrity = calculate_structural_integrity(
    aoi=None,
    edge_depth_m=EDGE_DEPTH_M,
    neighborhood_m=NEIGHBORHOOD_M,
    scale_m=SCALE_M,
)

# Add metadata
structural_integrity = structural_integrity.toFloat().set(
    {
        "edge_depth_m": EDGE_DEPTH_M,
        "neighborhood_m": NEIGHBORHOOD_M,
        "resolution_m": SCALE_M,
        "created_date": datetime.datetime.now().strftime("%Y-%m-%d"),
        "description": "Quality-weighted core area structural integrity",
    }
)

print("✓ Structural integrity graph built (lazy)")
print(f"  Band: {structural_integrity.bandNames().getInfo()}")

--- Step 1: Ensure Pre-computed Grid Exists ---
=== Creating Pre-computed Global Grid ===
Tile size: 10.0°
Minimum land percentage: 1.0%
Grid already exists at: projects/landler-open-data/assets/eii/intermediate/functional/grid/global_grid_10deg
Use overwrite=True to recreate it.


## Step 2: Export Global Asset 

In [ ]:
try:
    existing = ee.data.getAsset(OUTPUT_ASSET_PATH)
    print(f"⚠️  Asset already exists: {OUTPUT_ASSET_PATH}")
    print("   Delete it first to overwrite.")
    RUN_EXPORT = False
except ee.EEException:
    RUN_EXPORT = True

if RUN_EXPORT:
    print(f"Exporting structural integrity to: {OUTPUT_ASSET_PATH}")
    print(f"Region: Global ({GLOBAL_BOUNDS.bounds().getInfo()['coordinates']})")
    print(f"Scale: {SCALE_M}m")
    print()
    print("NOTE: This is a single export task. GEE handles internal tiling.")
    print("      Expected runtime: 1-4 hours depending on GEE load.")

    task = ee.batch.Export.image.toAsset(
        image=structural_integrity,
        description="Structural_Integrity_Global",
        assetId=OUTPUT_ASSET_PATH,
        region=GLOBAL_BOUNDS,
        scale=SCALE_M,
        crs="EPSG:4326",
        maxPixels=1e13,
        pyramidingPolicy={".default": "mean"},
    )
    task.start()

    print(f"\n✓ Export task started: {task.id}")
    print("  Monitor at: https://code.earthengine.google.com/tasks")

--- Step 3: Load Processing Grid ---
=== Loading Pre-computed Global Grid ===
Loading grid from: projects/landler-open-data/assets/eii/intermediate/functional/grid/global_grid_10deg
Filtered for land tiles only
Filtered for tiles >= -60° latitude
Filtered for tiles <= 80° latitude
Loaded 262 tiles
Total land tiles to process: 262


In [ ]:
# Verify output (run after export completes)
try:
    result = ee.Image(OUTPUT_ASSET_PATH)
    print("=== Output Verification ===")
    print(f"Asset: {OUTPUT_ASSET_PATH}")
    print(f"Band: {result.bandNames().getInfo()}")
    print(f"Properties: {result.propertyNames().getInfo()}")

    # Sample at a test point
    sample_point = ee.Geometry.Point([-60, -10])  # Amazon
    sample = result.sample(region=sample_point, scale=SCALE_M).first().getInfo()
    print(f"\nSample value (Amazon): {sample['properties']['structural_integrity']:.3f}")
except ee.EEException as e:
    print(f"Asset not yet available: {e}")


=== Structural Integrity Tiled Export ===
Processing 1 tiles at 300m resolution
Edge depth: 300m, Neighborhood: 5000m
Output folder: projects/landler-open-data/assets/eii/products/v1/structural_integrity/core_area
Processing tile 1/1: tile_-70_-10 (Land: 98.3%)

=== Summary ===
Total tiles: 1
Skipped (already exist): 0
Export tasks started: 1
Failed tiles: 0
